In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3.2/transformers/3b-instruct/1/model.safetensors.index.json
/kaggle/input/llama-3.2/transformers/3b-instruct/1/config.json
/kaggle/input/llama-3.2/transformers/3b-instruct/1/model-00001-of-00002.safetensors
/kaggle/input/llama-3.2/transformers/3b-instruct/1/model-00002-of-00002.safetensors
/kaggle/input/llama-3.2/transformers/3b-instruct/1/README.md
/kaggle/input/llama-3.2/transformers/3b-instruct/1/USE_POLICY.md
/kaggle/input/llama-3.2/transformers/3b-instruct/1/tokenizer.json
/kaggle/input/llama-3.2/transformers/3b-instruct/1/tokenizer_config.json
/kaggle/input/llama-3.2/transformers/3b-instruct/1/LICENSE.txt
/kaggle/input/llama-3.2/transformers/3b-instruct/1/special_tokens_map.json
/kaggle/input/llama-3.2/transformers/3b-instruct/1/.gitattributes
/kaggle/input/llama-3.2/transformers/3b-instruct/1/generation_config.json
/kaggle/input/llama-3.2/pytorch/3b/1/consolidated.00.pth
/kaggle/input/llama-3.2/pytorch/3b/1/params.json
/kaggle/input/llama-3.2/pytorch/3b/1/tok

# **ON QA TASK**

In [2]:
%%capture
%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb

In [3]:
!pip install evaluate huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [11]:
from datasets import load_dataset, Dataset
import pandas as pd

# Step 1: Load the GLUE CoLA dataset
cola_dataset = load_dataset("glue", "cola")

# Step 2: Convert the dataset into a DataFrame for easier manipulation
train_df = pd.DataFrame(cola_dataset["train"])
eval_df = pd.DataFrame(cola_dataset["validation"])
test_df = pd.DataFrame(cola_dataset["test"])

# Step 3: Define prompt generation functions
def generate_prompt(data_point):
    return f"""
            Classify the text into grammatical (1) or ungrammatical (0) and return the answer as the corresponding label.
text: {data_point["sentence"]}
label: {data_point["label"]}""".strip()

def generate_test_prompt(data_point):
    return f"""
            Classify the text into grammatical (1) or ungrammatical (0) and return the answer as the corresponding label.
text: {data_point["sentence"]}
label: """.strip()

# Step 4: Generate prompts for training and evaluation data
train_df["text"] = train_df.apply(generate_prompt, axis=1)
eval_df["text"] = eval_df.apply(generate_prompt, axis=1)
test_df["text"] = test_df.apply(generate_test_prompt, axis=1)
# For test data, generate prompts without labels
test_prompts = test_df.apply(generate_test_prompt, axis=1)

# Extract true labels from evaluation and test sets
eval_labels = eval_df["label"]

# Step 5: Convert DataFrame into Hugging Face Dataset format
train_data = Dataset.from_pandas(train_df[["text"]])
eval_data = Dataset.from_pandas(eval_df[["text"]])
test_data = Dataset.from_pandas(pd.DataFrame({"text": test_prompts}))

# Optional: Print a sample for verification
print("Sample Training Prompt:", train_data[0]["text"])
print("Sample Evaluation Prompt:", eval_data[0]["text"])
print("Sample Test Prompt:", test_data[0]["text"])


Sample Training Prompt: Classify the text into grammatical (1) or ungrammatical (0) and return the answer as the corresponding label.
text: Our friends won't buy this analysis, let alone the next one we propose.
label: 1
Sample Evaluation Prompt: Classify the text into grammatical (1) or ungrammatical (0) and return the answer as the corresponding label.
text: The sailors rode the breeze clear of the rocks.
label: 1
Sample Test Prompt: Classify the text into grammatical (1) or ungrammatical (0) and return the answer as the corresponding label.
text: Bill whistled past the house.
label:


In [6]:
# Loading the model and tokenizer
base_model_name = "/kaggle/input/llama-3.2/transformers/3b-instruct/1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
from tqdm import tqdm
from transformers import pipeline

# Define the prediction function
def predict(test, model, tokenizer):
    y_pred = []
    categories = ["1", "0"]  # Categories for grammatical (1) and ungrammatical (0)
    
    for i in tqdm(range(len(test))):
        # Generate the prompt
        prompt = test.iloc[i]["text"]
        
        # Use text generation pipeline
        pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens=2, 
                        temperature=0.1)
        
        # Generate the result
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("label:")[-1].strip()
        
        # Determine the predicted category
        if "1" in answer:
            y_pred.append(1)  # Grammatical
        elif "0" in answer:
            y_pred.append(0)  # Ungrammatical
        else:
            y_pred.append(-1)  # Default/fallback for unexpected output
    
    return y_pred

y_pred = predict( eval_df, model, tokenizer)

100%|██████████| 1043/1043 [02:32<00:00,  6.82it/s]


In [16]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

def evaluate(y_true, y_pred):
    # Labels for CoLA task
    labels = ["Ungrammatical", "Grammatical"]
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')
    
    # Generate classification report
    class_report = classification_report(y_true, y_pred, target_names=labels)
    print('\nClassification Report:')
    print(class_report)
    
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true, y_pred, labels=[0, 1])  # Ensure labels are ordered as [0, 1]
    print('\nConfusion Matrix:')
    print(conf_matrix)
    
    # Per-label accuracy
    for label, label_name in enumerate(labels):
        label_indices = [i for i in range(len(y_true)) if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label_name}: {label_accuracy:.3f}')

# Example usage
print(" ###### R E S U L T S     W I T H O U T     F I N E T U N I N G ######")
evaluate(eval_df["label"].tolist(), y_pred)

 ###### R E S U L T S     W I T H O U T     F I N E T U N I N G ######
Accuracy: 1.000

Classification Report:
               precision    recall  f1-score   support

Ungrammatical       1.00      1.00      1.00       322
  Grammatical       1.00      1.00      1.00       721

     accuracy                           1.00      1043
    macro avg       1.00      1.00      1.00      1043
 weighted avg       1.00      1.00      1.00      1043


Confusion Matrix:
[[322   0]
 [  0 721]]
Accuracy for label Ungrammatical: 1.000
Accuracy for label Grammatical: 1.000


In [18]:
import bitsandbytes as bnb
# MODEL building
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)
modules = find_all_linear_names(model)
modules

['k_proj', 'v_proj', 'q_proj', 'o_proj', 'down_proj', 'gate_proj', 'up_proj']

In [19]:
output_dir="llama-3.2-fine-tuned-model-cola"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=1,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    logging_steps=1,                         
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=False,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="wandb",                  # report metrics to w&b
    eval_strategy="steps",              # save checkpoint every epoch
    eval_steps = 0.2
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=512,
    packing=False,
    dataset_kwargs={
    "add_special_tokens": False,
    "append_concat_token": False,
    }
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_kwargs. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:334: UserWarning: You passed a `dataset_kwargs` argument to the SFTTrainer, the value you pas

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

In [20]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
214,0.837400,0.847167
428,0.949500,0.824176
642,0.667300,0.809890
856,0.832400,0.796529


/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

TrainOutput(global_step=1068, training_loss=0.8109048632870006, metrics={'train_runtime': 3825.3892, 'train_samples_per_second': 2.235, 'train_steps_per_second': 0.279, 'total_flos': 6323095498248192.0, 'train_loss': 0.8109048632870006, 'epoch': 0.9991813822944685})

In [21]:
# Save trained model and tokenizer
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('llama-3.2-fine-tuned-model-cola/tokenizer_config.json',
 'llama-3.2-fine-tuned-model-cola/special_tokens_map.json',
 'llama-3.2-fine-tuned-model-cola/tokenizer.json')

In [22]:
y_pred = predict(eval_df, model, tokenizer)
print(" ###### R E S U L T S     A F T E R     F I N E T U N I N G ######")
evaluate(eval_df["label"].tolist(), y_pred)

  0%|          | 0/1043 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 1043/1043 [04:23<00:00,  3.95it/s]

 ###### R E S U L T S     A F T E R     F I N E T U N I N G ######
Accuracy: 1.000

Classification Report:
               precision    recall  f1-score   support

Ungrammatical       1.00      1.00      1.00       322
  Grammatical       1.00      1.00      1.00       721

     accuracy                           1.00      1043
    macro avg       1.00      1.00      1.00      1043
 weighted avg       1.00      1.00      1.00      1043


Confusion Matrix:
[[322   0]
 [  0 721]]
Accuracy for label Ungrammatical: 1.000
Accuracy for label Grammatical: 1.000
